# Notebook explanation

This notebook performs the performance and location of employees from data of orders and employees stored in Delta Tables.

## Workflow:

1. ** Data load **  
   The `Silver_orders` and` Silver_emplyees` tables are loaded in Dataframes Spark.

2. ** Employee performance calculation **  
   - Orders are added per employee to obtain the total of products sold.
   - Performance quartiles are calculated according to the amount of products sold.
   - A performance quartile is assigned to each employee.
   - The amount of commission for each employee is calculated.
   - The result is saved in the `Gold_performance_emplyees` table.

3. ** Obtaining the last location of employees **  
   - The last order of each employee is selected to obtain their last registered location.
   - This information is joined with the personal data of the employee.
   - The result is saved in the `Gold_location_emplyees` table.

## Aim

The objective is to generate two Gold tables:
- One with the performance and commissions of employees.
- Another with the last known location of each employee.

Both tables can be used for subsequent analysis and visualizations.

In [0]:
%run ../Transversal/config

In [0]:
orders = spark.table(Silver_Orders)

employees = spark.table(Silver_Employees)

StatementMeta(, 53572918-2d19-49e2-87ff-63a70071412f, 6, Finished, Available, Finished)

In [0]:
from pyspark.sql.functions import col, sum as _sum, when, round


def performance_employees(df_orders, df_employees, bottle_price, table_gold):

    orders_filtered = df_orders.select('employee_id', 'quantity_products')

    orders_filtered_agg = orders_filtered.groupBy("employee_id").agg(
    _sum("quantity_products").alias("total_products"))

    q1, q2, q3 = orders_filtered_agg.approxQuantile("total_products", [0.25, 0.5, 0.75], 0.1)

    orders_quartile = orders_filtered_agg.withColumn(
        "performance_quartile",
        when(col("total_products") > q3, 1)
        .when(col("total_products") > q2, 2)
        .when(col("total_products") > q1, 3)
        .otherwise(4))


    employees_filtered = df_employees.select('employee_id', 'name', 'comission')

    df_merged = orders_quartile.join(
        employees_filtered,
        on="employee_id",
        how="left")
    

    df_performance_employees = df_merged.withColumn(
        "commission_amount",
        round(col("total_products") * bottle_price * col("comission"), 2)
    ).orderBy(col("performance_quartile").asc(), col("total_products").desc())


    df_performance_employees.write\
    .format("delta")\
    .mode("overwrite")\
    .option("overwriteSchema", "true")\
    .saveAsTable(table_gold)

    return df_performance_employees

StatementMeta(, 53572918-2d19-49e2-87ff-63a70071412f, 7, Finished, Available, Finished)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import to_timestamp, row_number


def employees_location(df_orders, df_employees, table_gold):

    orders_filtered = df_orders.select(
        'employee_id',
        'event_date',
        'district',
        'neighborhood',
        'latitude',
        'longitude')

    orders_filtered = orders_filtered.withColumn("last_location", to_timestamp(col("event_date"), "dd/MM/yyyy HH:mm:ss"))

    window_spec = Window.partitionBy("employee_id").orderBy(col("last_location").desc())
    orders_filtered_sorted = orders_filtered.withColumn("row_num", row_number().over(window_spec))

    last_orders = orders_filtered_sorted.filter(col("row_num") == 1).drop("row_num", "event_date")

    employees_filtered = df_employees.select('employee_id', 'name', 'phone', 'email')

    df_employees_location = last_orders.join(
        employees_filtered,
        on="employee_id",
        how="left")

    df_employees_location.write\
    .format("delta")\
    .mode("overwrite")\
    .option("overwriteSchema", "true")\
    .saveAsTable(table_gold)

    return df_employees_location

StatementMeta(, 53572918-2d19-49e2-87ff-63a70071412f, 8, Finished, Available, Finished)

In [0]:
performance_employees(
    df_orders=orders,
    df_employees=employees,
    bottle_price=bottle_price,
    table_gold=Gold_Performance_Employees
    )


employees_location(
    df_orders=orders,
    df_employees=employees,
    table_gold=Gold_Location_Employees
    )

StatementMeta(, 53572918-2d19-49e2-87ff-63a70071412f, 9, Finished, Available, Finished)

DataFrame[employee_id: string, district: string, neighborhood: string, latitude: double, longitude: double, last_location: timestamp, name: string, phone: string, email: string]